<a href="https://colab.research.google.com/github/LeoFeitosaP/Text-to-speech/blob/main/text_to_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gtts playsound pyttsx3 wikipedia

In [ ]:
from IPython.display import display, Audio, clear_output
import ipywidgets as widgets
import wikipedia
import webbrowser
from datetime import datetime
from gtts import gTTS
import os
import time

In [ ]:
class ColabAssistant:
    def __init__(self):
        # Configuração dos comandos
        self.wake_word = "assistente"
        self.commands = {
            "wikipedia": ["pesquisar na wikipedia", "procure na wikipedia", "o que é"],
            "youtube": ["abrir youtube", "pesquisar no youtube"],
            "pharmacy": ["farmácia mais próxima", "onde tem uma farmácia"],
            "time": ["que horas são", "hora atual"],
            "date": ["que dia é hoje", "data de hoje"],
            "stop": ["parar", "encerrar", "sair"]
        }

        # Configuração da interface
        self.setup_ui()

    def setup_ui(self):
        """Configura os elementos da interface no Colab"""
        self.output = widgets.Output()

        # Área de entrada de texto
        self.text_input = widgets.Text(
            placeholder='Digite seu comando (ex: "assistente que horas são")',
            layout=widgets.Layout(width='80%')
        )

        # Botão de enviar
        self.submit_button = widgets.Button(description="Enviar Comando")
        self.submit_button.on_click(self.on_submit)

        # Botão para falar hora/data
        self.time_button = widgets.Button(description="Que horas são?")
        self.time_button.on_click(lambda b: self.process_command("assistente que horas são"))

        # Botão para Wikipedia
        self.wiki_button = widgets.Button(description="Pesquisar Wikipedia")
        self.wiki_button.on_click(lambda b: self.process_command("assistente pesquisar na wikipedia " + self.text_input.value))

        display(widgets.VBox([
            widgets.Label("Digite seu comando começando com 'assistente':"),
            self.text_input,
            self.submit_button,
            widgets.HBox([self.time_button, self.wiki_button]),
            self.output
        ]))

    def on_submit(self, b):
        """Handler para o botão de enviar"""
        with self.output:
            clear_output()
            command = self.text_input.value
            if command:
                self.process_command(command)

    def speak(self, text):
        """Converte texto em áudio usando gTTS e reproduz no Colab"""
        with self.output:
            print(f"Assistente: {text}")

            # Cria o arquivo de áudio temporário
            tts = gTTS(text=text, lang='pt-br')
            tts.save("output.mp3")

            # Reproduz o áudio
            display(Audio("output.mp3", autoplay=True))

            # Limpa o arquivo temporário após 3 segundos
            time.sleep(3)
            os.remove("output.mp3")

    def process_command(self, command):
        """Processa o comando digitado"""
        if not command.lower().startswith(self.wake_word):
            self.speak(f"Por favor, comece seu comando com '{self.wake_word}'")
            return

        # Remove a palavra de ativação
        clean_command = command[len(self.wake_word):].strip().lower()

        # Processa os comandos
        if any(cmd in clean_command for cmd in self.commands["wikipedia"]):
            self.handle_wikipedia(clean_command)
        elif any(cmd in clean_command for cmd in self.commands["youtube"]):
            self.handle_youtube(clean_command)
        elif any(cmd in clean_command for cmd in self.commands["pharmacy"]):
            self.handle_pharmacy()
        elif any(cmd in clean_command for cmd in self.commands["time"]):
            self.handle_time()
        elif any(cmd in clean_command for cmd in self.commands["date"]):
            self.handle_date()
        elif any(cmd in clean_command for cmd in self.commands["stop"]):
            self.speak("Encerrando a assistente. Até mais!")
            return
        else:
            self.speak("Desculpe, não entendi esse comando. Pode repetir?")

    def handle_wikipedia(self, command):
        """Processa pesquisa na Wikipedia"""
        search_term = command
        for trigger in self.commands["wikipedia"]:
            search_term = search_term.replace(trigger, "").strip()

        try:
            self.speak(f"Pesquisando na Wikipedia sobre {search_term}")
            wikipedia.set_lang("pt")
            result = wikipedia.summary(search_term, sentences=2)
            self.speak(f"Segundo a Wikipedia: {result}")
        except wikipedia.exceptions.DisambiguationError as e:
            self.speak("Há várias opções para essa pesquisa. Por favor, seja mais específico.")
        except wikipedia.exceptions.PageError:
            self.speak("Não encontrei informações sobre esse tópico na Wikipedia.")

    def handle_youtube(self, command):
        """Abre o YouTube ou pesquisa nele"""
        search_term = command
        for trigger in self.commands["youtube"]:
            search_term = search_term.replace(trigger, "").strip()

        if search_term:
            url = f"https://www.youtube.com/results?search_query={search_term}"
            self.speak(f"Pesquisando no YouTube por {search_term}")
        else:
            url = "https://www.youtube.com"
            self.speak("Abrindo o YouTube")

        # Tenta abrir em nova aba (pode não funcionar em alguns navegadores)
        try:
            webbrowser.open(url, new=2)
        except:
            self.speak(f"Você pode acessar manualmente: {url}")

    def handle_pharmacy(self):
        """Localiza farmácias próximas (simulação)"""
        self.speak("Estou verificando as farmácias mais próximas de você.")
        self.speak("Em uma implementação real, eu usaria sua localização para encontrar farmácias próximas.")

    def handle_time(self):
        """Informa a hora atual"""
        now = datetime.now()
        current_time = now.strftime("%H:%M")
        self.speak(f"Agora são {current_time}")

    def handle_date(self):
        """Informa a data atual"""
        months = [
            "janeiro", "fevereiro", "março", "abril", "maio", "junho",
            "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"
        ]
        today = datetime.now()
        date_str = f"{today.day} de {months[today.month-1]} de {today.year}"
        self.speak(f"Hoje é {date_str}")


In [ ]:
assistant = ColabAssistant()